In [4]:
import pandas as pd
comments=pd.read_excel('evaluations_overall_comments.xlsx')

In [1]:
!pip install transformers

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [6]:


# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiff2/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
# labels=[]
# mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
# with urllib.request.urlopen(mapping_link) as f:
#     html = f.read().decode('utf-8').split("\n")
#     csvreader = csv.reader(html, delimiter='\t')
# labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

comments['processed']=comments['Answer:'].apply(lambda x: preprocess(x))
comments['encoded']=comments['processed'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output']=comments['encoded'].apply(lambda x:model(**x))
comments['scores']=comments['output'].apply(lambda x:softmax(x[0][0].detach().numpy()))
print(comments)

         Target:                      Evaluator:       Rotation Dates:  \
0     Eliza Chen      Dr. Ong, Andrew Ming Liang    7/1/2022-7/31/2022   
1     Eliza Chen         Dr. Chan, Webber Pak Wo    7/1/2022-7/31/2022   
2     Eliza Chen              Dr. Lim, Chee Hooi    7/1/2022-7/31/2022   
3     Eliza Chen         Dr. Chang, Jason Pik Eu    8/1/2022-8/31/2022   
4     Eliza Chen              Dr. Tan, Chee Kiat    8/1/2022-8/31/2022   
..           ...                             ...                   ...   
163  Rachel Yeap     Dr. Tan, Malcolm Teck Kiang    7/1/2022-7/31/2022   
164  Rachel Yeap  Dr. Khor, Christopher Jen Lock    8/1/2022-8/31/2022   
165  Rachel Yeap               Dr. Liou, Wei Lun    8/1/2022-8/31/2022   
166  Rachel Yeap             Dr. Loo, Khang Ning    9/1/2022-10/2/2022   
167  Rachel Yeap        Dr. Kwek, Andrew Boon Eu  10/3/2022-10/31/2022   

                         Service:  \
0                             - -   
1       SHS-GASTRO:BASIC ENDO-SGH   


In [5]:
# NEUTRAL COMMENTS: 
comments['neu']=comments['scores'].apply(lambda x: x[1])
comments[['Answer:','neu']].loc[comments['neu']>0.7]

,Answer:,neu
2,No concern,0.744895
5,Please see comments above,0.864153
18,No concern,0.744895
20,able to act independently in most of the clini...,0.840395
50,Able to work almost independently.n,0.704185
56,"Keen to learn, follows through on tasks, techn...",0.769361
89,Knowledge aspect needs further improvement,0.700757
113,As per earlier comments,0.849452
122,We had discussed previously about Andre's 5 pe...,0.742418
124,Andre would need to show more dedication to le...,0.708658


In [9]:
# NEGATIVE COMMENTS
comments['neg']=comments['scores'].apply(lambda x: x[0])
comments[['Answer:','neg']].loc[comments['neg']>0.2].values.tolist()

[['Had a short week with her, hence limited interactions.', 0.28766068816185],
 ['Xiuhuan cares for her patients and that\'s evident from how she greets them as people when doing the round (as opposed to patients). However she needs to apply herself more. Although she is gaining the skills and knowledge requisite of her level, she needs to remember that she is training to be a consultant and not a SR. She sometimes "hedges" her diagnosis and this approach will make her less effective in the long run.',
  0.4184384047985077],
 ['enthusiastic but sometimes may lost focus', 0.37071284651756287],
 ['He needs to be more proactive in learning . He is quite passive in learning and asking senior.',
  0.22275735437870026],
 ['I think overall whilst Andre is not a malignant person, he requires supervision to complete his tasks. With regards to his leave practices, to give him the benefit of the doubt, he "might not have known" and he certainly has needs for his family, however the lack of consid

In [8]:
# NON POSITIVE COMMENTS
comments['pos']=comments['scores'].apply(lambda x: x[2])
comments[['Answer:','pos']].loc[comments['pos']<0.2].values.tolist()

[['No concern', 0.07848860323429108],
 ['Eliza Chen has a quiet disposition and gives good thoughts to her management. However, because she is a bit too quiet, she does not project the impression that she have good control over her subordinates. There were knowledge gaps as we discuss cases - she is aware of them and has insight.',
  0.19857768714427948],
 ['Please see comments above', 0.04341280087828636],
 ['No concern', 0.07848860323429108],
 ['Had a short week with her, hence limited interactions.',
  0.032620709389448166],
 ['able to act independently in most of the clinical situation',
  0.08608968555927277],
 ['Keen to learn, follows through on tasks, technical competencies adequate for level of training, takes effort to reflect on learning points',
  0.18451350927352905],
 ['Xiuhuan cares for her patients and that\'s evident from how she greets them as people when doing the round (as opposed to patients). However she needs to apply herself more. Although she is gaining the skil

In [16]:
MODEL = f"trained-model"

tokenizer = AutoTokenizer.from_pretrained('cardiff2/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

comments['processed2']=comments['Answer:'].apply(lambda x: preprocess(x))
comments['encoded2']=comments['processed2'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output2']=comments['encoded2'].apply(lambda x:model(**x))
comments['scores2']=comments['output2'].apply(lambda x:softmax(x[0][0].detach().numpy()))
print(comments)

         Target:                      Evaluator:       Rotation Dates:  \
0     Eliza Chen      Dr. Ong, Andrew Ming Liang    7/1/2022-7/31/2022   
1     Eliza Chen         Dr. Chan, Webber Pak Wo    7/1/2022-7/31/2022   
2     Eliza Chen              Dr. Lim, Chee Hooi    7/1/2022-7/31/2022   
3     Eliza Chen         Dr. Chang, Jason Pik Eu    8/1/2022-8/31/2022   
4     Eliza Chen              Dr. Tan, Chee Kiat    8/1/2022-8/31/2022   
..           ...                             ...                   ...   
163  Rachel Yeap     Dr. Tan, Malcolm Teck Kiang    7/1/2022-7/31/2022   
164  Rachel Yeap  Dr. Khor, Christopher Jen Lock    8/1/2022-8/31/2022   
165  Rachel Yeap               Dr. Liou, Wei Lun    8/1/2022-8/31/2022   
166  Rachel Yeap             Dr. Loo, Khang Ning    9/1/2022-10/2/2022   
167  Rachel Yeap        Dr. Kwek, Andrew Boon Eu  10/3/2022-10/31/2022   

                         Service:  \
0                             - -   
1       SHS-GASTRO:BASIC ENDO-SGH   


In [ ]:
comments['scores3']=comments['scores2'].apply(lambda x:x[1])
comments['scores3'].values.tolist()

In [7]:
MODEL = f"self-labelled-trained"

tokenizer = AutoTokenizer.from_pretrained('cardiff2/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

comments['processed3']=comments['Answer:'].apply(lambda x: preprocess(x))
comments['encoded3']=comments['processed3'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output3']=comments['encoded3'].apply(lambda x:model(**x))
comments['scores3']=comments['output3'].apply(lambda x:softmax(x[0][0].detach().numpy()))
print(comments)

         Target:                      Evaluator:       Rotation Dates:  \
0     Eliza Chen      Dr. Ong, Andrew Ming Liang    7/1/2022-7/31/2022   
1     Eliza Chen         Dr. Chan, Webber Pak Wo    7/1/2022-7/31/2022   
2     Eliza Chen              Dr. Lim, Chee Hooi    7/1/2022-7/31/2022   
3     Eliza Chen         Dr. Chang, Jason Pik Eu    8/1/2022-8/31/2022   
4     Eliza Chen              Dr. Tan, Chee Kiat    8/1/2022-8/31/2022   
..           ...                             ...                   ...   
163  Rachel Yeap     Dr. Tan, Malcolm Teck Kiang    7/1/2022-7/31/2022   
164  Rachel Yeap  Dr. Khor, Christopher Jen Lock    8/1/2022-8/31/2022   
165  Rachel Yeap               Dr. Liou, Wei Lun    8/1/2022-8/31/2022   
166  Rachel Yeap             Dr. Loo, Khang Ning    9/1/2022-10/2/2022   
167  Rachel Yeap        Dr. Kwek, Andrew Boon Eu  10/3/2022-10/31/2022   

                         Service:  \
0                             - -   
1       SHS-GASTRO:BASIC ENDO-SGH   


In [8]:
# NEUTRAL COMMENTS: 
comments['neu']=comments['scores3'].apply(lambda x: x[1])
comments[['Answer:','neu']].loc[comments['neu']>0.7]

,Answer:,neu
2,No concern,0.786172
18,No concern,0.786172
86,To pick up on some research projects to expand...,0.761624
113,As per earlier comments,0.771054


In [14]:
# NON POSITIVE COMMENTS
comments['pos']=comments['scores3'].apply(lambda x: x[2])
comments[['Answer:','pos']].loc[comments['pos']<0.2].values.tolist()

[['No concern', 0.06928350031375885],
 ['Eliza Chen has a quiet disposition and gives good thoughts to her management. However, because she is a bit too quiet, she does not project the impression that she have good control over her subordinates. There were knowledge gaps as we discuss cases - she is aware of them and has insight.',
  0.09634342789649963],
 ['Please see comments above', 0.02324148453772068],
 ['No concern', 0.06928350031375885],
 ['Had a short week with her, hence limited interactions.',
  0.018455933779478073],
 ['able to act independently in most of the clinical situation',
  0.0395875945687294],
 ['Able to work almost independently.n', 0.14367574453353882],
 ['Keen to learn, follows through on tasks, technical competencies adequate for level of training, takes effort to reflect on learning points',
  0.05060121417045593],
 ['She is very introspective and seems to give a lot of thought and effort to making the clinical diagnosis or management plan. There are instances

In [56]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import datasets
from sklearn.metrics import f1_score

import torch
import numpy as np

# Load pre-trained question answering model
model_name = "distilbert-base-cased-distilled-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load validation or test dataset
dataset = datasets.load_dataset("squad")

# Loop through validation or test examples and generate predictions
predictions = []
for example in dataset["validation"]:
    question = example["question"]
    context = example["context"]
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    start_logits, end_logits = model(**inputs)
    start_logits_tensor = torch.from_numpy(np.fromstring(start_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
    end_logits_tensor = torch.from_numpy(np.fromstring(end_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
    if start_logits_tensor.numel() == 0 or end_logits_tensor.numel() == 0:
        # If either tensor is empty, append an empty string to predictions
        predictions.append("")
    else:
        start_index = torch.argmax(start_logits_tensor)
        end_index = torch.argmax(end_logits_tensor)
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))
        predictions.append(answer)

# Calculate F1 score
f1 = f1_score(dataset["validation"]["answers"]["text"], predictions, average="macro")

print(f"F1 score: {f1}")


Found cached dataset squad (C:/Users/feryo/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 166.56it/s]
C:\Users\feryo\AppData\Local\Temp\ipykernel_3996\2059637340.py:23: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  start_logits_tensor = torch.from_numpy(np.fromstring(start_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
C:\Users\feryo\AppData\Local\Temp\ipykernel_3996\2059637340.py:24: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  end_logits_tensor = torch.from_numpy(np.fromstring(end_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
Token indices sequence length is longer than the specified maximum sequence length for this model (637 > 512). Running this sequence through the model will result in indexing errors

RuntimeError: The size of tensor a (637) must match the size of tensor b (512) at non-singleton dimension 1

In [58]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import datasets
from sklearn.metrics import f1_score

import torch
import numpy as np

# Load pre-trained question answering model
model_name = "distilbert-base-cased-distilled-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load validation or test dataset
dataset = datasets.load_dataset("squad")

# Loop through validation or test examples and generate predictions
predictions = []
for example in dataset["validation"]:
    question = example["question"]
    context = example["context"]
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, max_length=512, return_tensors="pt")
    start_logits, end_logits = model(**inputs)
    start_logits_tensor = torch.from_numpy(np.fromstring(start_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
    end_logits_tensor = torch.from_numpy(np.fromstring(end_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
    if start_logits_tensor.numel() == 0 or end_logits_tensor.numel() == 0:
        # If either tensor is empty, append an empty string to predictions
        predictions.append("")
    else:
        start_index = torch.argmax(start_logits_tensor)
        end_index = torch.argmax(end_logits_tensor)
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))
        predictions.append(answer)

# Calculate F1 score
f1 = f1_score(dataset["validation"]["answers"]["text"], predictions, average="macro")

print(f"F1 score: {f1}")


Found cached dataset squad (C:/Users/feryo/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 167.11it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\feryo\AppData\Local\Temp\ipykernel_3996\1568049786.py:23: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  start_logits_tensor = torch.from_numpy(np.fromstring(start_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
C:\Users\feryo\AppData\Local\Temp\ipykernel_3996\1568049786.py:24: DeprecationWarning: string or file could not 

TypeError: list indices must be integers or slices, not str

In [68]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import datasets
from sklearn.metrics import f1_score,accuracy_score

import torch
import numpy as np

# Load pre-trained question answering model
model_name = "deepset/roberta-base-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load validation or test dataset
dataset = datasets.load_dataset("squad")

# Get validation examples
validation_examples = dataset["validation"]

# Loop through validation examples and generate predictions
predictions = []
for example in validation_examples:
    question = example["question"]
    context = example["context"]
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, max_length=512, return_tensors="pt")
    start_logits, end_logits = model(**inputs)
    start_logits_tensor = torch.from_numpy(np.fromstring(start_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
    end_logits_tensor = torch.from_numpy(np.fromstring(end_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
    if start_logits_tensor.numel() == 0 or end_logits_tensor.numel() == 0:
            # If either tensor is empty, append an empty string to predictions
            
        predictions.append("")
    else:
        start_index = torch.argmax(start_logits_tensor)
        end_index = torch.argmax(end_logits_tensor)
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))
        predictions.append(answer)

# Get validation answers
validation_answers = [example["answers"]["text"][0] for example in validation_examples]

# Calculate F1 score
f1 = f1_score(validation_answers, predictions, average="macro")

print(f"F1 score: {f1}")


Found cached dataset squad (C:/Users/feryo/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 167.38it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\feryo\AppData\Local\Temp\ipykernel_3996\1891880838.py:26: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  start_logits_tensor = torch.from_numpy(np.fromstring(start_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
C:\Users\feryo\AppData\Local\Temp\ipykernel_3996\1891880838.py:27: DeprecationWarning: string or file could not 

F1 score: 0.0


In [70]:
print(predictions)

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [66]:
from sklearn.metrics import f1_score,accuracy_score

In [67]:
f1 = accuracy_score(validation_answers, predictions, average="macro")
print(f1)

TypeError: got an unexpected keyword argument 'average'

In [71]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import datasets

import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Load pre-trained question answering model
model_name = "distilbert-base-cased-distilled-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load validation or test dataset
dataset = datasets.load_dataset("squad")

# Get validation or test examples
examples = dataset["validation"]  # or dataset["test"]

# Initialize variables for counting correct and total predictions
num_exact_match = 0
num_correct = 0
total_predictions = 0
all_ground_truths = []
all_predictions = []

# Loop through examples and generate predictions
for example in examples:
    question = example["question"]
    context = example["context"]
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, max_length=512, return_tensors="pt")
    start_logits, end_logits = model(**inputs)
    start_logits_tensor = torch.from_numpy(np.fromstring(start_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
    end_logits_tensor = torch.from_numpy(np.fromstring(end_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
    start_index = torch.argmax(start_logits_tensor)
    end_index = torch.argmax(end_logits_tensor)
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))
    prediction = answer.strip().lower()  # convert to lowercase and remove leading/trailing whitespace
    ground_truths = [x["text"].strip().lower() for x in example["answers"]]  # convert to lowercase and remove leading/trailing whitespace
    all_ground_truths.append(ground_truths)
    all_predictions.append(prediction)
    if prediction in ground_truths:
        num_correct += 1
    if prediction == ground_truths[0]:
        num_exact_match += 1
    total_predictions += 1

# Calculate metrics
exact_match = num_exact_match / total_predictions
precision = precision_score(all_ground_truths, all_predictions, average="macro")
recall = recall_score(all_ground_truths, all_predictions, average="macro")
f1 = f1_score(all_ground_truths, all_predictions, average="macro")

print(f"Exact Match: {exact_match}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")


Found cached dataset squad (C:/Users/feryo/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 182.27it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\feryo\AppData\Local\Temp\ipykernel_3996\336092189.py:32: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  start_logits_tensor = torch.from_numpy(np.fromstring(start_logits, dtype=np.float32, sep=" ")).unsqueeze(0)
C:\Users\feryo\AppData\Local\Temp\ipykernel_3996\336092189.py:33: DeprecationWarning: string or file could not be

IndexError: argmax(): Expected reduction dim to be specified for input.numel() == 0.

: 